In [6]:
import numpy as np                  
import matplotlib.pyplot as plt

# This function is used to add two matrices together without exceeding the pixel value of 255.
def add_to_255(a, b):
    """Return a+b to a max value of 255"""
    if a + b > 255:
        return 255
    else:
        return a + b
v_add_to_255 = np.vectorize(add_to_255)  # Here we create a "vectorized" form of our simple function above.

def noisy_line(img, label):
    """Adds background noise and a vertical line on the right edge to match data"""
    rng = np.random.default_rng()  # This function uses a random number generator, created here.
    line = np.zeros((28,28,1), dtype=np.uint8)  # Our line will be defined in a 28x28 matrix, initial set to zeroes.
    
    # We pick a random number c, from 20 to 25, as the center position of our horizontal line.
    c = rng.integers(20, high=25, size=1)[0]  # randomized position of line
    a = rng.integers(0, high=100, size=1)[0]  # randomized transparency of line

    # We set the center of the line to a max of 200 and adjacent rows to a max of 100.
    line[c,:] = (100 - a)
    line[c+1,:] = (200 - 2 * a)
    line[c+2,:] = (100 - a)
    noise = rng.integers(0, high=(100-a),size=(3,28,1))
    line[c:c+3,:] = line[c:c+3,:] - noise
    img = v_add_to_255(img, line)
    return img, label

# Some examples of the noise we will add to EMNIST
# z = np.zeros((28,28, 1), dtype=np.uint8)
# view(noisy_line(z, 0)[0])  # The views show normalized luminence, i.e. they treat the highest value found as white.
# view(noisy_line(z, 1)[0])  # This means that our transparency value has no impact in these previews of the noisy lines.
# view(noisy_line(z, 2)[0])

# # Some examples of EMNIST letters with noise added.
# plt.figure(figsize=(8, 8))
# for i, (image, label) in enumerate(ds_train_readable.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     #img_numpy = image.numpy()[..., None]  # Adding a dimension to match TF dataset API
#     img_numpy = image.numpy()  # Adding a dimension to match TF dataset API
#     img, lbl = noisy_line(img_numpy, label.numpy())
#     plt.imshow(img)
#     plt.title(str(int(lbl)))
#     plt.axis("off")


In [23]:
import tensorflow_datasets as tfds
import tensorflow as tf
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print(ds_info)

# tfds.show_statistics(ds_info) Not working
labels, counts = np.unique(np.fromiter(ds_train.map(lambda x, y: y), np.int32), return_counts=True)
pos = counts[23]
neg = 88800 - pos
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
initial_bias = np.log([pos/neg])
print(f'Initial bias: {initial_bias}')

ds_train_readable = ds_train.map(
  lambda img, label: (tf.transpose(img, perm=[1,0,2]), tf.cast([(label == 23)], tf.int64)),
  num_parallel_calls=tf.data.AUTOTUNE,
  deterministic=True)

ds_train_readable_noisy = ds_train_readable.map(
  lambda img, label: tf.numpy_function(func=noisy_line, inp=[img, label], Tout=(tf.uint8, tf.int64)),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=False)

def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label
ds_train_readable_noisy_float = ds_train_readable_noisy.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

def set_shapes(image, label):
  image.set_shape([28, 28, 1])
  label.set_shape([1])
  return image, label
ds_train_final = ds_train_readable_noisy_float.map(set_shapes, num_parallel_calls=tf.data.AUTOTUNE)

# Some additional dataset setup steps
ds_train_final = ds_train_final.cache()
ds_train_final = ds_train_final.batch(400)  # This changes the shape of the data, so call it after all mapped functions..
ds_train_final = ds_train_final.prefetch(tf.data.AUTOTUNE)
print("final element_spec", ds_train_final.element_spec)

# Then we need to apply the same functions and settings to the test dataset
ds_test_readable = ds_test.map(
  lambda img, label: (tf.transpose(img, perm=[1,0,2]), tf.cast([(label == 23)], tf.int64)),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=True)

ds_test_readable_noisy = ds_test_readable.map(
  lambda img, label: tf.numpy_function(func=noisy_line, inp=[img, label], Tout=(tf.uint8, tf.int64)),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=False)
ds_test_readable_noisy_float = ds_test_readable_noisy.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test_final = ds_test_readable_noisy_float.map(set_shapes, num_parallel_calls=tf.data.AUTOTUNE)
ds_test_final = ds_test_final.batch(400)
ds_test_final = ds_test_final.cache()
ds_test_final = ds_test_final.prefetch(tf.data.AUTOTUNE)
print("final test element_spec", ds_test_final.element_spec)

tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/letters/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST Letters
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_dir='/home/jansen/tensorflow_datasets/emnist/letters/3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=44.14 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=37),
    }),
    supervis

2024-05-13 12:34:23.303798: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
from tensorflow import keras

METRICS = [
      keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      keras.metrics.MeanSquaredError(name='Brier score'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_dense_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    model = keras.Sequential([
        keras.Input(shape=(28,28), name='input'),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=metrics)

    return model

In [25]:
dense_model = make_dense_model(output_bias=initial_bias)

dense_model.fit(
    ds_train_final,BinaryCrossentropy
    epochs=15,
    validation_data=ds_test_final,
)

Epoch 1/15


2024-05-13 12:34:31.427446: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


222/222 ━━━━━━━━━━━━━━━━━━━━ 64s 283ms/step - Brier score: 0.0159 - accuracy: 0.9799 - auc: 0.9582 - cross entropy: 0.0611 - fn: 602.2960 - fp: 186.6143 - loss: 0.0611 - prc: 0.7527 - precision: 0.8222 - recall: 0.5675 - tn: 42896.7188 - tp: 1112.5785 - val_Brier score: 0.0018 - val_accuracy: 0.9982 - val_auc: 0.0000e+00 - val_cross entropy: 0.0097 - val_fn: 0.0000e+00 - val_fp: 26.0000 - val_loss: 0.0097 - val_prc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 14774.0000 - val_tp: 0.0000e+00
Epoch 2/15
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - Brier score: 0.0079 - accuracy: 0.9895 - auc: 0.9913 - cross entropy: 0.0304 - fn: 324.8520 - fp: 124.9910 - loss: 0.0304 - prc: 0.9281 - precision: 0.9115 - recall: 0.7995 - tn: 42958.3398 - tp: 1390.0225 - val_Brier score: 7.9752e-04 - val_accuracy: 0.9995 - val_auc: 0.0000e+00 - val_cross entropy: 0.0053 - val_fn: 0.0000e+00 - val_fp: 8.0000 - val_loss: 0.0053 - val_prc: 0.0000e+00 - val_precision: 0.0000e+00 - va

## Using a Convolutional Neural Network (CNN)

In [26]:
def make_conv_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    model = tf.keras.models.Sequential([
        keras.Input(shape=(28, 28, 1)),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation="sigmoid", bias_initializer=output_bias)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=metrics)

    return model


In [27]:
conv_model = make_conv_model(output_bias=initial_bias)
conv_model.fit(
    ds_train_final,
    epochs=15,
    validation_data=ds_test_final,
)

Epoch 1/15
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - Brier score: 0.0124 - accuracy: 0.9845 - auc: 0.9712 - cross entropy: 0.0482 - fn: 779.9910 - fp: 166.6861 - loss: 0.0777 - prc: 0.6512 - precision: 0.7519 - recall: 0.4315 - tn: 57716.6445 - tp: 934.8834 - val_Brier score: 8.2666e-04 - val_accuracy: 0.9992 - val_auc: 0.0000e+00 - val_cross entropy: 0.0055 - val_fn: 0.0000e+00 - val_fp: 12.0000 - val_loss: 0.0055 - val_prc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 14788.0000 - val_tp: 0.0000e+00
Epoch 2/15
222/222 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - Brier score: 0.0070 - accuracy: 0.9915 - auc: 0.9941 - cross entropy: 0.0265 - fn: 264.6278 - fp: 100.8251 - loss: 0.0265 - prc: 0.9421 - precision: 0.9319 - recall: 0.8359 - tn: 42982.5078 - tp: 1450.2466 - val_Brier score: 6.7269e-04 - val_accuracy: 0.9993 - val_auc: 0.0000e+00 - val_cross entropy: 0.0042 - val_fn: 0.0000e+00 - val_fp: 11.0000 - val_loss: 0.0042 - val_prc: 0.0000e+00 - val_precision: 0

In [28]:
dense_model.save("dense_model_2.keras", overwrite=True)
conv_model.save("conv_model_2.keras", overwrite=True)
